# Phase 4: Running BEAST

<details>
    <summary>Click To See A Decription of Parameters</summary>
        <pre>
            <code>
save_dir: str  
    Path to directory for saving outputs in.

number_of_beast_runs: int
    Number of chains to use (repeated runs to do) when running BEAST.

seeds: list of ints
    Seeds to use when running BEAST.

beast_ram: str
    RAM to use for each run of beast see sbatch documentation.

beast_threads: int
    Threads/CPUs to use for each run of beast see https://www.beast2.org/2021/03/31/command-line-options.html.

max_jobs: int, default 12
    Maximum number of cores to use at one time.

  </code>
</pre>


In [ ]:
save_dir=''
seeds=''
beast_arg_string=''
jobs_per_time=1
optimisation_test=False
beast_xml_path=''
kernel='beast_pype'

In [ ]:
if [ "$optimisation_test" == "False" ] && [ "$beast_xml" == "" ]
then
    beast_xml_path=$save_dir/beast.xml
fi

In [ ]:
# For some reason this still has to be called so that slurm's sbatch to use the beast_pype environment.
source activate $kernel

In [ ]:
if [ -e $beast_xml_path ]
    then
    mkdir $save_dir/beast_outputs
    parallel --jobs=$jobs_per_time --results $save_dir/beast_outputs/run-with-seed-{1}.out \
        beast \
        -seed {1} \
        -prefix $save_dir/beast_outputs/run-with-seed-{1}- \
        -statefile $save_dir/beast_outputs/run-with-seed-{1}.state \
        $beast_arg_string \
        $beast_xml_path \
    ::: ${seeds[@]}
    else
    valid_dirs=$(find $save_dir -maxdepth 1 -type d)
    array_to_remove=($save_dir $save_dir/.ipynb_checkpoints)
    for element in "${array_to_remove[@]}"; do valid_dirs=("${valid_dirs[@]/$element}"); done
    parallel --jobs=$jobs_per_time mkdir {}/beast_outputs ::: ${valid_dirs[@]}
    parallel --jobs=$jobs_per_time --link --results {1}/beast_outputs/run-with-seed-{2}.out \
        beast \
        -seed {2} \
        -prefix {1}/beast_outputs/run-with-seed-{2}- \
        -statefile {1}/beast_outputs/run-with-seed-{2}.state \
        $beast_arg_string \
        {1}/beast.xml \
    ::: ${valid_dirs[@]} ::: ${seeds[@]}
fi